In [1]:
from neo4j import GraphDatabase
import xlwings as xw
from uuid import uuid4

In [2]:
wb = xw.Book('Curriculum Links export.xlsx')
sheet = wb.sheets[0]

In [3]:
program_level_learning_outcomes = []

In [4]:
def append_to_program_level_outcomes(start_range, end_range, program_level_category, uid_start):
    plo_uid = uid_start
    
    for i in range(start_range, end_range):
        
        program_level_learning_outcomes.append({
            "category": program_level_category,
            "uid": plo_uid,
            "title": sheet[f"B{i}"].value,
            "description": sheet[f"C{i}"].value,
            "created_at": "datetime()", 
            "updated_at": "datetime()"
        })
        
        plo_uid += 1

In [5]:
append_to_program_level_outcomes(32, 44, "Certificate Learning Outcomes", 1)
append_to_program_level_outcomes(49, 57, "Diploma Learning Outcomes", 13)
append_to_program_level_outcomes(62, 63, "Master's Learning Outcomes", 21)

In [6]:
courses = []

In [7]:
for i in range(108, 543):
    if sheet[f"A{i}"].value is None:
        i += 1
        continue


    if "----------" in sheet[f"A{i}"].value:
        course = {
            "uid": str(uuid4()),
            "code": "DATA",
            "number": sheet[f"A{i}"].value.split(' ')[3],
            "title": sheet[f"A{i}"].value.split('  ')[1].split(' -')[0].strip(),
            "created_at": "datetime()", 
            "updated_at": "datetime()",
            "course_outcomes": []
        }


        i += 3
        course_outcome_i = 1
        while sheet[f"A{i}"].value is not None and "Course Outcome" in sheet[f"A{i}"].value:
            course["course_outcomes"].append({
                "uid": course_outcome_i,
                "description": sheet[f"B{i}"].value.replace('"', ''),
                "created_at": "datetime()", 
                "updated_at": "datetime()",
                "outcome_maps": []
            })
            
            course_outcome_i += 1         
            i += 1

            
        i += 4
        outcome_map_i = 1
        while sheet[f"A{i}"].value is not None and "Course Outcome" in sheet[f"A{i}"].value:
            outcome_map_list = []
            for index, letter in enumerate(list("BCDEFGHIJKLMNOPQRSTUV")):
                if sheet[f"{letter}{i}"].value == 'Y':
                    outcome_map_list.append(index + 1)
                    
            course_outcome_to_map_to = next(x for x in course["course_outcomes"] if x["uid"] == outcome_map_i)
            course_outcome_to_map_to["outcome_maps"] = outcome_map_list
            i += 1
            outcome_map_i += 1
                        
        courses.append(course)

In [8]:
courses

[{'id': 'fc5c7636-9513-4de4-a49a-fe3a12af1706',
  'code': 'DATA',
  'number': '601',
  'title': 'Working with Data and Visualization',
  'created_at': 'datetime()',
  'updated_at': 'datetime()',
  'course_outcomes': [{'uid': 1,
    'description': 'Use programmatic tools and suitable programming paradigms to clean and transform data into a format suitable for subsequent analytics',
    'created_at': 'datetime()',
    'updated_at': 'datetime()',
    'outcome_maps': [1, 6, 9]},
   {'uid': 2,
    'description': 'Use existing software libraries to summarize and aggregate structured data',
    'created_at': 'datetime()',
    'updated_at': 'datetime()',
    'outcome_maps': [6, 7, 9]},
   {'uid': 3,
    'description': 'Use existing software libraries to explore data and create informative and engaging data visualizations',
    'created_at': 'datetime()',
    'updated_at': 'datetime()',
    'outcome_maps': [3, 4, 5, 9]},
   {'uid': 4,
    'description': 'Present data stories in an interactive a

In [9]:
# with GraphDatabase.driver("neo4j+s://d8794294.databases.neo4j.io",
#                           auth=("neo4j", "bmY2FfIhsCZKM1KUDjzSlvdDLWmmp-BpfmReQn3Cl9U")) as driver:
#     driver.verify_connectivity()

In [10]:
create_courses_query = ""

In [11]:
for course in courses:
    course_var = f"c{course['number']}"
    create_courses_query += f"CREATE ({course_var}:Course {{uid: '{str(uuid4())}', code: 'DATA', number: {course['number']}, title: '{course['title']}', created_at: datetime(), updated_at: datetime()}})\n"
    
    for course_outcome in course["course_outcomes"]:
        course_outcome_var = f"{course_var}_{course_outcome['uid']}"
        create_courses_query += f"CREATE ({course_outcome_var}:CourseOutcome {{uid: '{str(uuid4())}', description: \"{course_outcome['description']}\", created_at: datetime(), updated_at: datetime() }})\n"
        
        for outcome_map_id in course_outcome["outcome_maps"]:
            plo = next(x for x in program_level_learning_outcomes if x["uid"] == outcome_map_id)
            create_courses_query += f"CREATE ({course_outcome_var})-[:MAPS_TO {{uid: '{str(uuid4())}', created_at: datetime(), updated_at: datetime()}}]->(:ProgramOutcome {{uid: '{str(uuid4())}', category: \"{plo['category']}\", title: '{plo['title']}', description: '{plo['description']}', created_at: datetime(), updated_at: datetime()}})\n"
    
    create_courses_query += f"\n"

In [12]:
print(create_courses_query)

CREATE (c601:Course {uid: '39fdf6cb-7a0e-4478-85bc-28436cbc9a0e', code: 'DATA', number: 601, title: 'Working with Data and Visualization', created_at: datetime(), updated_at: datetime()})
CREATE (c601_1:CourseOutcome {uid: '451aae7a-0fcd-4bef-a60d-0c1568bb2055', description: "Use programmatic tools and suitable programming paradigms to clean and transform data into a format suitable for subsequent analytics", created_at: datetime(), updated_at: datetime() })
CREATE (c601_1)-[:MAPS_TO {uid: '80e9d242-9d8f-466c-93fa-f40ab98687da', created_at: datetime(), updated_at: datetime()}]->(:ProgramOutcome {uid: 'e4de574a-bcfc-4e56-8c6a-040d959f79eb', category: "Certificate Learning Outcomes", title: 'Data Organization', description: 'Understand, compare, and utilize a variety of different data organization strategies, data types, and storage formats.', created_at: datetime(), updated_at: datetime()})
CREATE (c601_1)-[:MAPS_TO {uid: '54189fb8-8cd8-44a4-ab8b-975e161ec1bf', created_at: datetime(), u

In [13]:
course_prerequisites = {
    603: [602],
    604: [601],
    605: [601, 602, 603, 604],
    606: [601, 602, 603, 604],
    607: [601, 602, 603, 604],
    608: [601, 602, 603, 604],
    # 609: [601, 602, 603, 604], commented because 609 and 610 are not in the curriculum links doc
    # 610: [601, 602, 603, 604],
    611: [601, 602, 603, 604],
    612: [601, 602, 603, 604],
    613: [601, 602, 603, 604],
    614: [601, 602, 603, 604, 611],
    621: [601, 602, 603, 604],
    622: [601, 602, 603, 604],
    623: [601, 602, 603, 604],
    624: [601, 602, 603, 604],
    693: [691],
    695: [691],
}

In [14]:
create_course_prereqs_query = ""
matched_course_numbers = []

In [15]:
for course_number, course_prereqs in course_prerequisites.items():
    course_var = f"c{course_number}"
    if course_number not in matched_course_numbers:
        create_course_prereqs_query += f"MATCH ({course_var}:Course {{number: {course_number} }})\n"
        matched_course_numbers.append(course_number)
    
    for course_prereq_number in course_prereqs:
        course_prereq_var = f"c{course_prereq_number}"
        if course_prereq_number not in matched_course_numbers:
            create_course_prereqs_query += f"MATCH ({course_prereq_var}:Course {{number: {course_prereq_number} }})\n" 
            matched_course_numbers.append(course_prereq_number)

create_course_prereqs_query += f"WITH *\n"
        
for course_number, course_prereqs in course_prerequisites.items():
    course_var = f"c{course_number}"
    
    for course_prereq_number in course_prereqs:
        course_prereq_var = f"c{course_prereq_number}"     
        # create_course_prereqs_query += f"CREATE ({course_var})-[:HAS_PREREQUISITE]->({course_prereq_var})\n"
        create_course_prereqs_query += f"CREATE ({course_prereq_var})-[:IS_PREREQUISITE_OF {{uid: '{str(uuid4())}',created_at: datetime(), updated_at: datetime()}}]->({course_var})\n"

In [16]:
print(create_course_prereqs_query)

MATCH (c603:Course {number: 603 })
MATCH (c602:Course {number: 602 })
MATCH (c604:Course {number: 604 })
MATCH (c601:Course {number: 601 })
MATCH (c605:Course {number: 605 })
MATCH (c606:Course {number: 606 })
MATCH (c607:Course {number: 607 })
MATCH (c608:Course {number: 608 })
MATCH (c611:Course {number: 611 })
MATCH (c612:Course {number: 612 })
MATCH (c613:Course {number: 613 })
MATCH (c614:Course {number: 614 })
MATCH (c621:Course {number: 621 })
MATCH (c622:Course {number: 622 })
MATCH (c623:Course {number: 623 })
MATCH (c624:Course {number: 624 })
MATCH (c693:Course {number: 693 })
MATCH (c691:Course {number: 691 })
MATCH (c695:Course {number: 695 })
WITH *
CREATE (c602)-[:IS_PREREQUISITE_OF {uid: 'edccb87a-0bb9-4dc6-9189-93d2d6aef777',created_at: datetime(), updated_at: datetime()}]->(c603)
CREATE (c601)-[:IS_PREREQUISITE_OF {uid: '5aef85ab-d609-4602-9785-b9f479cc5f46',created_at: datetime(), updated_at: datetime()}]->(c604)
CREATE (c601)-[:IS_PREREQUISITE_OF {uid: 'd47a98ec-042

In [17]:
# AI generated
knowledge_areas = [
    {
        "title": "Foundational Mathematics & Statistics",
        "description": "Linear algebra, calculus, probability theory, statistics"
    },
    {
        "title": "Programming Languages",
        "description": "Python, R, SQL"
    },
    {
        "title": "Data Wrangling & Manipulation",
        "description": "Data cleaning, transformation, manipulation"
    },
    {
        "title": "Machine Learning",
        "description": "Supervised learning, unsupervised learning, deep learning"
    },
    {
        "title": "Data Visualization",
        "description": "Communicating insights through charts, graphs, etc."
    },
    {
        "title": "Big Data & Cloud Computing",
        "description": "Big data technologies, cloud computing platforms"
    },
    {
        "title": "Domain Knowledge",
        "description": "Knowledge of the specific field (finance, healthcare, etc.)"
    }
]

In [18]:
create_knowledge_areas_query = ''

In [19]:
for knowledge_area in knowledge_areas:
    create_knowledge_areas_query += f"CREATE (:KnowledgeArea {{uid: '{str(uuid4())}', title: '{knowledge_area['title']}', description: '{knowledge_area['description']}', created_at: datetime(), updated_at: datetime()}})\n"

In [20]:
print(create_knowledge_areas_query)

CREATE (:KnowledgeArea {uid: '56f56785-0ce3-4711-8b73-7a2b09a69409', title: 'Foundational Mathematics & Statistics', description: 'Linear algebra, calculus, probability theory, statistics', created_at: datetime(), updated_at: datetime()})
CREATE (:KnowledgeArea {uid: '0c11b03e-3cf3-483f-ba2b-edc32689278e', title: 'Programming Languages', description: 'Python, R, SQL', created_at: datetime(), updated_at: datetime()})
CREATE (:KnowledgeArea {uid: '618c071c-0836-4714-b09a-951c0987e5f8', title: 'Data Wrangling & Manipulation', description: 'Data cleaning, transformation, manipulation', created_at: datetime(), updated_at: datetime()})
CREATE (:KnowledgeArea {uid: 'ac296918-d3a6-4f78-afb5-c659b43c6f2d', title: 'Machine Learning', description: 'Supervised learning, unsupervised learning, deep learning', created_at: datetime(), updated_at: datetime()})
CREATE (:KnowledgeArea {uid: '01a4c654-e67f-4a7d-84cf-34fb567047ee', title: 'Data Visualization', description: 'Communicating insights through 

In [24]:
topic_name_unique_constraint = '''
    CREATE CONSTRAINT topic_name_unique IF NOT EXISTS
    FOR (t:Topic)
    REQUIRE t.name IS UNIQUE
'''
course_number_unique_constraint = '''
    CREATE CONSTRAINT course_number_unique IF NOT EXISTS
    FOR (c:Course)
    REQUIRE c.number IS UNIQUE
'''
knowledge_area_unique_constraint = '''
    CREATE CONSTRAINT knowledge_area_unique IF NOT EXISTS
    FOR (ka:KnowledgeArea)
    REQUIRE ka.title IS UNIQUE
'''

In [26]:
# auradb
# url = 
# password = 

# localhost
password = "kQPBOJQGbECDNwlrFgulekcHVOGkGRPNAfCAPdZhdrI"
url = "neo4j://localhost:7687"

In [28]:
with GraphDatabase.driver(url,
                          auth=("neo4j", password)) as driver:
    driver.execute_query(create_courses_query, database_="neo4j")
    driver.execute_query(create_course_prereqs_query, database_="neo4j")
    driver.execute_query(create_knowledge_areas_query, database_="neo4j")
    driver.execute_query(topic_name_unique_constraint, database_="neo4j")
    driver.execute_query(course_number_unique_constraint, database_="neo4j")
    driver.execute_query(knowledge_area_unique_constraint, database_="neo4j")